In [1]:
import contextlib
import datetime as dt
import io
import itertools as it
import logging; logging.basicConfig(level=logging.DEBUG)
import types
import warnings
import zipfile

import conda_helpers as ch
import lxml
import mpm
import mpm.api
import mpm.ui.gtk
import openpyxl as ox
import openpyxl.worksheet.datavalidation
import pandas as pd
import path_helpers as ph
import xlsxwriter
import yaml
reload(ox)
reload(ox.worksheet)
reload(ox.reader.worksheet)
reload(mpm)
reload(mpm.api)
reload(mpm.ui.gtk)
reload(ch)


EXCEL_NAMESPACES = {k: getattr(ox.xml.constants, k)
                    for k in ('SHEET_MAIN_NS', 'REL_NS', 'PKG_REL_NS')}

In [2]:
def load_extension_lists(xlsx_path):
    '''
    Load extension list for each worksheet in an Excel spreadsheet.
    
    Extension lists include, e.g., worksheet data validation settings.
    
    Note that ``openpyxl`` does not currently `support reading existing data
    validation <http://openpyxl.readthedocs.io/en/default/validation.html#validating-cells>`_.
    As a workaround, this function makes it possible to load the extension
    list for each worksheet in a workbook so they may be restored to a
    workbook modified by ``openpyxl`` using  :func:`update_extension_lists`.
    
    See also
    --------
    :func:`update_extension_lists`
    
    Parameters
    ----------
    xlsx_path : str
        Path to Excel ``xlsx`` file.
    
    Returns
    -------
    dict
        Mapping from each worksheet filepath in Excel ZIP file to
        corresponding extension list XML element (or ``None`` if the
        worksheet does not contain any extension list element).
    '''
    extension_lists = {}

    # Open Excel file.
    with zipfile.ZipFile(xlsx_path, mode='r') as input_:
        # Get mapping from each worksheet filename to corresponding `ZipInfo`
        # object.
        zip_info_by_filenames = {ph.path(v.filename): v
                                 for v in input_.filelist}
        
        # Extract extension list XML element (if present) from each worksheet.
        for filename_i, zip_info_i in zip_info_by_filenames.iteritems():
            if filename_i.parent != 'xl/worksheets':
                continue
            with io.BytesIO(input_.read(filename_i)) as data_i:
                worksheet_root_i = lxml.etree.parse(data_i)
            extension_lists_i = worksheet_root_i.xpath('/SHEET_MAIN_NS:worksheet'
                                                       '/SHEET_MAIN_NS:extLst',
                                                       namespaces=
                                                       EXCEL_NAMESPACES)
            if extension_lists_i:
                # This worksheet has an extension list.
                extension_lists[filename_i] = extension_lists_i[0]
            else:
                extension_lists[filename_i] = None
    return extension_lists


def update_extension_lists(xlsx_path, extension_lists):
    '''
    Update extension list for worksheets in an Excel spreadsheet.
    
    Extension lists include, e.g., worksheet data validation settings.
    
    Note that ``openpyxl`` does not currently `support reading existing data
    validation <http://openpyxl.readthedocs.io/en/default/validation.html#validating-cells>`_.
    As a workaround, this function makes it possible to restore extension
    lists saved using :func:`load_extension_lists` after modifying the
    workbook with ``openpyxl``.
    
    See also
    --------
    :func:`load_extension_lists` 
    
    Parameters
    ----------
    xlsx_path : str
        Path to Excel ``xlsx`` file.
    extension_lists : dict
        Mapping from each worksheet filepath in Excel ZIP file to
        corresponding extension list XML element.
    '''
    with io.BytesIO() as output:
        with zipfile.ZipFile(output, mode='a',
                             compression=zipfile.ZIP_DEFLATED) as output_zip:
            # - Read existing file
            # - Append extension list from template file to worksheet XML.
            # - Copy all files except for `worksheet1` to in-memory zip file.
            with zipfile.ZipFile(xlsx_path, mode='r') as input_:
                zip_infos_by_filename = {ph.path(v.filename): v
                                         for v in input_.filelist}
                for filename_i, zip_info_i in zip_infos_by_filename.iteritems():
                    extension_list_i = extension_lists.get(filename_i)
                    
                    if extension_list_i is None or filename_i not in extension_lists:
                        # Worksheet file has no extension list.  Use original
                        # worksheet contents.
                        contents_i = input_.read(filename_i)
                    else:
                        # Worksheet file has **extension list**.
                        # Load worksheet file XML contents from `xlsx_path` file.
                        with io.BytesIO(input_.read(filename_i)) as data:
                            root_i = lxml.etree.parse(data)
                        # Get root worksheet XML element.
                        worksheet_i = root_i.xpath('/SHEET_MAIN_NS:worksheet',
                                                   namespaces=EXCEL_NAMESPACES)[0]
                        # Append the extension list to the worksheet element.
                        worksheet_i.append(extension_list_i)
                        # Use modified worksheet contents with extension list
                        # added.
                        contents_i = lxml.etree.tostring(root_i)
                    # Write worksheet contents to output zip.
                    output_zip.writestr(filename_i, contents_i,
                                        zip_info_i.compress_type)
            output_zip.close()
        return output.getvalue()


def get_column_widths(worksheet, min_width=None):
    '''
    Parameters
    ----------
    worksheet : openpyxl.worksheet.worksheet.Worksheet
        Excel worksheet.
    min_width : int, optional
        Minimum column width in characters.
        
    Returns
    -------
    dict
        Mapping from letter of each column containing at least one non-blank
        cell to the corresponding column width to fit the widest entry in the
        column.
    '''
    key = lambda x: x.column

    column_widths =  {column_i: max(max(len(str(cell_ij.value))
                                        for cell_ij in cells_i),
                                    min_width or 0)
                      for column_i, cells_i in
                      it.groupby(sorted(worksheet.get_cell_collection(),
                                        key=key), key=key)}
    return column_widths

In [3]:
output_path = ph.path('output.xlsx')
template_path = ph.path(r'..\templates\DRC Data Collection.xlsx')

extension_lists = load_extension_lists(template_path)

with warnings.catch_warnings():
    warnings.filterwarnings('ignore', 'Data Validation extension is not '
                            'supported and will be removed', UserWarning)
    
    with contextlib.closing(ox.load_workbook(template_path)) as workbook:
        workbook.active = 0
        worksheet = workbook.worksheets[0]
        worksheet.views.sheetView[0].selection[0].activeCell = 'A1'
        worksheet.views.sheetView[0].selection[0].sqref = 'A1'
        
        column_widths = get_column_widths(workbook.worksheets[1])
        column_width = max(column_widths.values())
        
        # Automatically adjust the width of columns to fit values.
        for column_i, width_i in get_column_widths(worksheet).iteritems():
            worksheet.column_dimensions[column_i].width = column_width
        workbook.save(output_path)

# Generate output Excel ZIP file binary contents with extension lists
# restored.
updated_xlsx = update_extension_lists(output_path, extension_lists)

with output_path.open('wb') as output:
    output.write(updated_xlsx)
output_path.launch()

In [ ]:
# Path to the template file containing entry choices for fields.
template_path = ph.path(r'..\templates\DRC Data Collection.xlsx')
# Read field values from Excel file.
df_field_values = pd.read_excel(template_path, sheetname='Field values')

field_choices = {k: v.dropna().astype(str).tolist()
                 for k, v in df_field_values.iteritems()}

## Styles
# Define style for date types.
date_style = ox.styles.NamedStyle(name='date', number_format='yyyy-mmm-dd')
# Define header font and border styles.
header_font = ox.styles.Font(bold=True)
thin_border = ox.styles.Side(border_style='thin')
header_border = ox.styles.Border(*(4 * [thin_border]))

## Create workbook with data validation for assay fields
# Create the workbook and worksheet we'll be working with
workbook = ox.Workbook()
worksheet = workbook.active

# Values to populate assay field cells (see "Fill assay field cell
# contents..." below).
#
# A value of `None` is used to indicate a place holder to ensure following
# values in the same row are placed in the appropriate column.
rows = [['Date'],
        ['Location'],
        ['Box ID'],
        ['Operator'],
        ['Device Type', None, 'Device ID'],
        ['Analyte', None, 'Multiplexed'],
        ['Standards']]

samples_header = ['Step ID Label', 'Sample ID', 'Sample Type', 'Accepted?',
                  'What Went Wrong? (Only if not accepted)', 'Notes']

styles = {'Date': date_style}

validators = {'Date': dict(type='date',
                           prompt='Please enter date of test (YYYY-MM-DD)',
                           error='Please enter date in format: YYYY-MM-DD',
                           errorTitle='Invalid date format',
                           allow_blank=False),
              'Device ID':
              dict(type='custom',
                   formula1=u'NOT(ISBLANK({cell}))',
                   prompt='Please enter device ID ',
                   error='Please enter device ID\n'
                   'Note: Cannot be empty',
                   errorTitle='Invalid Device ID')}

# Increase width of Assay field value columns to fit, e.g., full date.
for column_i in ('B', 'D'):
    worksheet.column_dimensions[column_i].width = 18

# Create dictionary to map each assay field name (e.g., "Date", "Location",
# etc.) to the Excel worksheet cell instance for the respective **label**.
#
# Note: the cell for the assay field **value** can be referenced by an
# **offset** from the assay field **label** cell (e.g.,
# `cell_map['Date'].offset(column=1)`).
cell_map = {}

# Fill assay field cell contents (starting from `A1`) with values in `rows`.
for i, record_i in enumerate(rows):
    for j, value_ij in enumerate(record_i):
        row_i = i + 1
        column_j = j + 1
        column_letter_j = ox.utils.get_column_letter(column_j)
        
        if value_ij is None:
            # No value to enter as cell contents.  Skip to next column.
            continue
        cell_ij = worksheet.cell(row=row_i, column=column_j, value=value_ij)
        # Set assay field label font and border style properties.
        cell_ij.font = header_font
        cell_ij.border = header_border
        # Add reference to the corresponding cell
        cell_map[value_ij] = cell_ij
        
assay_cells = sorted(worksheet.get_cell_collection(),
                     key=lambda x: (x.row, x.column))

# Place samples information header row two rows below the assay fields cells.
samples_header_row = assay_cells[-1].row + 2

for j, value_ij in enumerate(samples_header):
    row_i = samples_header_row + 1
    column_j = j + 1
    column_letter_j = ox.utils.get_column_letter(column_j)

    if value_ij is None:
        continue
    cell_ij = worksheet.cell(row=row_i, column=column_j, value=value_ij)
    cell_ij.font = header_font
    cell_ij.border = header_border
        
cell_map['Date'].offset(column=1).value = dt.datetime.utcnow().date()

for name_i, validator_i in validators.iteritems():
    if isinstance(validator_i, dict) and 'type' in validator_i:
        cell_i = cell_map[name_i].offset(column=1)
        kwargs_i = {k: v.format(cell=cell_i.coordinate) if isinstance(v, types.StringTypes) else v
                    for k, v in validator_i.iteritems()}
        validator_i = ox.worksheet.datavalidation.DataValidation(**kwargs_i)
    worksheet.add_data_validation(validator_i)
    cell_i = cell_map[name_i].offset(column=1)
    validator_i.add(cell_i)

for name_i, choices_i in field_choices.iteritems():
    # A list of choices exists exists for the current assay field name.
    #
    # If no data validator has been explicitly set for the assary field
    # **value cell**, create a data validator to display a **drop-down list**
    # containing the available choices.
    if name_i in cell_map and (name_i not in validators or
                               ('type' not in validators[name_i] and
                               'formula1' not in validators[name_i])):
        # For the current assay field either:
        #  - No custom validator has been explicitly assigned; or
        #  - Only some optional, non-type validator parameters have been
        #    assigned (e.g., `allow_blank=False, showErrorMessage=True`).
        
        # By default, allow user to type entry that is **not** in the choices
        # list.  This can be overridden by setting `showErrorMessage` to
        # `True` in the corresponding entry in the `validators` dictionary.
        base_validator_i =  dict(prompt='Please select {name}'
                                 .format(name=name_i),
                                 showErrorMessage=False,
                                 allow_blank=False)
        validator_i = base_validator_i.copy()
        validator_i.update(validators.get(name_i, {}))
        
        # Use "list" validator to show drop-down containing choices for field.
        validator_i['type'] = 'list'
        validator_i['formula1'] = '"{}"'.format(','.join(choices_i))
        validator_i = (ox.worksheet.datavalidation
                       .DataValidation(**validator_i))
        worksheet.add_data_validation(validator_i)
        cell_i = cell_map[name_i].offset(column=1)
        validator_i.add(cell_i)
            
for name_i, style_i in styles.iteritems():
    if name_i in cell_map:
        cell_map[name_i].offset(column=1).style = style_i

# Automatically adjust the width of columns to fit values.
for column_i, width_i in get_column_widths(worksheet,
                                           min_width=18).iteritems():
    worksheet.column_dimensions[column_i].width = width_i

test_path = ph.path('test.xlsx')
workbook.save(test_path)
test_path.launch()